In [ ]:
#
# Example - Using uvatradier to fetch quote data for a given symbol
#

import os
import dotenv
import pandas as pd
from uvatradier import Quotes

# Create .env file within current working directory.
# Add the following to your .env file. Specify the correct account number and authorization token for your quote.
# 	tradier_acct = <ACCOUNT_NUMBER>
#	tradier_token = <AUTH_TOKEN>

dotenv.load_dotenv()

ACCOUNT_NUMBER = os.getenv('tradier_acct')
AUTH_TOKEN = os.getenv('tradier_token')

# Initializing new Quotes object
quotes = Quotes(ACCOUNT_NUMBER, AUTH_TOKEN)

DOW_30_TICKER = [
    "AAPL",
    "MSFT",
    "JPM",
    "V",
    "RTX",
    "PG",
    "GS",
    "NKE",
    "DIS",
    "AXP",
    "HD",
    "INTC",
    "WMT",
    "IBM",
    "MRK",
    "UNH",
    #"KO",
    "CAT",
    "TRV",
    "JNJ",
    "CVX",
    "MCD",
    "VZ",
    "CSCO",
    "XOM",
    "BA",
    "MMM",
    "PFE",
    "WBA",
    "DD",
]

# data = pd.DataFrame(quotes.get_historical_quotes('KO', start_date='2022-10-01', end_date='2023-10-01'))

# for symb in DOW_30_TICKER:
#     data_to_append = pd.DataFrame(quotes.get_historical_quotes(symb, start_date='2022-10-01', end_date='2023-10-01'))
#     data = pd.concat([data, data_to_append], ignore_index=True)

data = pd.read_csv("DOW30data.csv")

print(data)

# data['close_change'] = data['close'].pct_change()
# data['volume_change'] = data['volume'].pct_change()\
close_change = data['close'].pct_change()
volume_change = data['volume'].pct_change();

In [ ]:
# import csv

# # Specify the file name
# file_name = 'DOW30data.csv'

# # Write the DataFrame to a CSV file
# data.to_csv(file_name, index=False)

# print(f'Data has been written to {file_name}')

In [ ]:
print(close_change)

In [ ]:
volume_change

In [ ]:
# Given the close_change Series (as per your example)
# close_change = pd.Series([None, 0.002295, -0.009510, -0.021515, -0.009449], name='close')  # truncated for brevity

# Replace NaN values with the mean of the non-NaN values
close_change_filled = close_change.fillna(close_change.mean())
volume_change_filled = volume_change.fillna(volume_change.mean())

# print(close_change_filled)
# print(volume_change_filled)


logit_data = pd.DataFrame({'close':close_change_filled, 'volume':volume_change_filled})
print(logit_data)


In [ ]:
logit_data.query("(volume > 10) or (volume < -10)")

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# # Assuming these are your Series (based on provided data)
# close = pd.Series([
#     -0.000009, 0.002295, -0.009510, -0.021515, -0.009449, # truncated for brevity
#     -0.010417, -0.008246, -0.010260, -0.002502, 0.003046
# ])

# volume = pd.Series([
#     0.057599, 0.158734, -0.240522, 0.200211, -0.332292, # truncated for brevity
#     -0.156129, -0.094194, 0.229716, -0.093092, -0.017077
# ])

# Convert close to binary. Assuming a positive change is 1 and non-positive is 0.
logit_data['close_binary'] = (logit_data['close'] > 0).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(logit_data[['volume']], logit_data['close_binary'], test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Get the score of the model
score = model.score(X_test, y_test)

print(f"Model Accuracy: {score*100:.2f}%")
